# Integrating machine breakdowns in the environment

- To manage the machine breakdowns within the environmnet, i have implemented the`` _check_machine_breakdowns(td)`` method in the environment.

### Documentation of the ``_check_machine_breakdowns(td)`` method

- INPUT: td
    - this method takes a tensordict representing the state of the environment as an input and follows following process

- Our main logic in here is that, we iterate over each machine in each batch
    - 1. we firstly get the breakdowns of the machine
      2. then we check all the machine breakdowns if this machine has a breakdown at the current time
      3. If machine has a breakdown
            - Then we mark machine busy_until the from this time till machine is repaired, therefore machine can not process any operation
            - Then we give the tensordict with the updated busy_until enties back

- Since, according to our assomptions a job cannot be preempted or interrupted,
- we need to check at each decision step whether a job has a machine breakdown or not

- In case a machine has a breakdown right at the beginning
    - we reset'e ekleseydik daha mi olurdu acba a

peki her decision step'inde bunu nasil kullanacagiz

# Manually chekcing if the ``_check_machine_breakdowns()`` method correct functions

## 1. Implementing the environment

In [2]:
from rl4co.envs.scheduling.djssp.env import DJSSPEnv
from rl4co.models import L2DPolicy, L2DModel
from rl4co.utils import RL4COTrainer
import gc
from rl4co.envs import JSSPEnv
from rl4co.models.zoo.l2d.model import L2DPPOModel
from rl4co.models.zoo.l2d.policy import L2DPolicy4PPO
from torch.utils.data import DataLoader
import json
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display, clear_output
import time
import networkx as nx
import matplotlib.pyplot as plt
from rl4co.envs import FJSPEnv
from rl4co.models.zoo.l2d import L2DModel
from rl4co.models.zoo.l2d.policy import L2DPolicy
from rl4co.models.zoo.l2d.decoder import L2DDecoder
from rl4co.models.nn.graph.hgnn import HetGNNEncoder
from rl4co.utils.trainer import RL4COTrainer
generator_params = {
"num_jobs" : 6 ,
"num_machines": 6 ,
"min_processing_time": 1 ,
"max_processing_time": 99 ,
"mtbf" : 17 ,
"mttr" : 4
}
env = DJSSPEnv(generator_params=generator_params,
_torchrl_mode=True,
stepwise_reward=True)

C:\Users\soner\anaconda3\envs\env_rl4co\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. Reset the environment and get the generated machine_breakdowns

In [3]:
td = env.reset(batch_size=[1])
td["machine_breakdowns"]

array([{0: [{'TIME': 3.000108480453491, 'DURATION': 10.950005531311035}, {'TIME': 35.32599139213562, 'DURATION': 1.3882921934127808}], 1: [{'TIME': 21.723852157592773, 'DURATION': 1.9373793601989746}, {'TIME': 53.32680654525757, 'DURATION': 1.5291308164596558}], 2: [{'TIME': 7.263213157653809, 'DURATION': 2.711613416671753}, {'TIME': 52.46433997154236, 'DURATION': 3.8198349475860596}], 3: [{'TIME': 40.77605056762695, 'DURATION': 1.205381989479065}, {'TIME': 45.75132215023041, 'DURATION': 0.6587216854095459}], 4: [{'TIME': 6.723081111907959, 'DURATION': 2.6919376850128174}, {'TIME': 38.85593867301941, 'DURATION': 0.6645970344543457}], 5: [{'TIME': 9.79523754119873, 'DURATION': 0.2698454260826111}, {'TIME': 25.17279475927353, 'DURATION': 8.416229248046875}, {'TIME': 46.51334512233734, 'DURATION': 3.2661590576171875}]}],
      dtype=object)

### Machine 0 has its first breakdown at time 3.000108480453491. However since the we do not check the machine breakdowns yet, our tensordict entry which indicates if machines are busy (td["busy_until"]) should be full with the 0's

In [4]:
td["busy_until"]

tensor([[0., 0., 0., 0., 0., 0.]])

## 3. After that to ensure our method works correctly
    We manually advance the time to the first machine breakdown occurence time and overwrite the tensordict

In [25]:
# advance the time
td["time"] = torch.Tensor([3.000108480453491])

# check the machine breakdowns and overwrite the tensordict
td = env._check_machine_breakdowns(td)

### Now, in td["busy_until"] entry -> machine 0 must be shown as busy until it is being repaired

In [24]:
td["busy_until"]

tensor([[13.9501,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000]])